## 机器学习：模型导入与数据导入

In [1]:
from pyspark.ml.recommendation import ALSModel
model_path = "hdfs:///user/erin/data/mllib/alsmodel"
als_model =ALSModel.load(model_path)
df=spark.sql("select * from user_erin.rating")
name=spark.read.csv('hdfs://default/user/erin/demo/movies_metadata.csv', header=True)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,None,pyspark,idle,Link,,erin,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user: integer (nullable = true)
 |-- movie: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- title: string (nullable = true)

In [3]:
name.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

从数据集中随机选择一个用户id，以便于后续进行检测。在limit（）的括号中填入选择的数量。

In [4]:
from pyspark.sql.functions import rand
df.orderBy(rand()).limit(1).select("user").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
|  user|
+------+
|203222|
+------+

## 机器学习：运用模型对用户进行推荐
- 对于als模型，可以用recommendForAllUsers（）方程，在括号中填入每个用户需要推荐的电影数量，例如以下单元格中使用10，那么对于每个用户，都会有10部电影推荐。
- recommendforuser（）这个方程的功能是将用户id对应的推荐的电影id转化为电影名称，使推荐更直观。

In [5]:
recommend=als_model.recommendForAllUsers(10)
recommend.select("recommendations").show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|     recommendations|
+--------------------+
|[[1789, 25.058905...|
+--------------------+
only showing top 1 row

In [6]:
def recommendforuser(userid):
    res=recommend.filter(recommend.user==userid)
    res=res.select("recommendations")
    res=res.take(1)[0]["recommendations"]
    tname=[]
    for i in res:
        temp=name.filter(name.id==i.movie)
        n=temp.select("title").take(1)[0]["title"]
        tname.append(n)
    return tname

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

推荐实例，在括号中填入用户id，则会得到一个包含十部电影名称的队列。
cell9中的用户id，是cell4输出的随机的一个用户id。

In [7]:
recommendforuser(21260)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Mai Stati Uniti', 'A Viking Saga: The Darkest Day', 'The Gazebo', 'Return Home', 'Blondie Johnson', '96.0', 'Let Me In', 'A Plasticine Crow', 'Cannibal Women in the Avocado Jungle of Death', 'Next Door']

将推荐电影数据集存入hive。

In [8]:
recommend.write.format("hive").mode("overwrite").saveAsTable("user_erin.recommend")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…